In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

c:\Users\Admin\OneDrive\Python


In [15]:
import numpy as np
import pandas as pd
import scipy
import toolkit as ftk
up = pd.Series([100,100.3,102.91,104.04,103.1,104.55,107.06,108.66,115.83,114.21,118.67,118.07,127.64,132.74,127.83,120.03,121.71,115.75,113.32,120.34,127.32,119.18,121.2,120.72,120.48,117.95,119.24,124.85,127.84,132.06,131.14,137.3,138.13,139.51,139.23,143.96,145.4
], index=pd.date_range('2000-01', periods=37, freq='M'))
rrr = ftk.price_to_return(up)

ftk.arithmetic_mean(up) - 0.011
ftk.compound_return(up) - 0.454
ftk.compound_return(up, annualize=True) - 0.1329
ftk.mean_abs_dev(up) - 0.0252
ftk.variance(up) - 0.0011 * 36 / 35


1.646978686484831e-07